# Day 17 - 3D Cellular Automaton in an infinite grid

* https://adventofcode.com/2020/day/17

More cellular automatons, now in 3 dimensions and with an infinite grid. We can use [`scipy.signal.convolve()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html) rather than `convolve2d()` we used before (see [day 18, 2018](../2018/Day%2018.ipynb), [day 24, 2019](../2019/Day%2024.ipynb) & [day 11 this year](./Day%2011.ipynb), but we have to extend our matrix each iteration.

To add 'edges' to the matrix, we can use [`numpy.pad()`](https://numpy.org/doc/stable/reference/generated/numpy.pad.html); we want to add zeros all round, so it's enough to use `numpy.pad(matrix, 1, 
=0)`.

In [1]:
from itertools import product
from enum import Enum

import numpy as np
from scipy.signal import convolve


class Cube(Enum):
    inactive = 0, '.'
    active = 1, '#'
    
    def __new__(cls, int_: int, value: str):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.int = int_
        return instance


class ConwayCubes:
    def __init__(self, initial_state: str, dimensions: int = 3) -> None:
        self._dimensions = dimensions
        extent = (1,) * (dimensions - 2)
        self._matrix = np.array([
            Cube(c).int for line in initial_state.splitlines() for c in line
        ]).reshape((-1, initial_state.index('\n'), *extent))

        # generate N-dimensional kernel; all 1s except the middle.
        self._kernel = np.full((3,) * dimensions, 1)
        self._kernel[(1,) * dimensions] = 0

    def __str__(self) -> str:
        letters = 'zw'
        m = self._matrix
        mapping = {s.int: s.value for s in Cube}
        layers, dims = [], m.shape[2:]
        for lidx in product(*map(range, dims)):
            layers.append(
                "\n".join(
                    [
                        " ".join([f"{c}={v - (d // 2)}" for c, v, d in zip(letters, lidx, dims)]),
                        *["".join(map(mapping.__getitem__, row)) for row in m[(Ellipsis, *lidx)]],
                    ]
                )
            )
        return '\n\n'.join(layers)

    def run(self, cycles: int = 6) -> int:
        """Run until stability is reached, the return the number of occupied seats"""
        f = self._matrix
        full = {c: np.full(self._matrix.shape, c.int) for c in Cube}
        for _ in range(cycles):
            # grow both the matrix, and the full matrices to use in np.select()
            f = np.pad(f, 1, constant_values=0)
            full = {c: np.pad(m, 1, constant_values=c.int) for c, m in full.items()}
            counts = {
                cube: convolve(f == cube.int, self._kernel, mode="same") for cube in Cube
            }
            rules = {
                # If a cube is **active** and **exactly 2 or 3** of its neighbors are also **active**,
                # the cube remains **active**. Otherwise, the cube becomes **inactive**.
                Cube.inactive: (
                    (f == Cube.active.int)
                    & ((counts[Cube.active] < 2) | (counts[Cube.active] > 3))
                ),
                # If a cube is **inactive** but **exactly 3** of its neighbors are active, the cube
                # becomes **active**. Otherwise, the cube remains **inactive**.
                Cube.active: (f == Cube.inactive.int) & (counts[Cube.active] == 3),
            }
            f = np.select(list(rules.values()), [full[c] for c in rules], default=f)
            self._matrix = f
        return np.sum(f == Cube.active.int)
 

teststate = ".#.\n..#\n###\n"
assert ConwayCubes(teststate).run() == 112

In [2]:
import aocd
initial_state = aocd.get_data(day=17, year=2020)

In [3]:
print("Part 1:", ConwayCubes(initial_state).run())

Part 1: 298


## Part 2, more dimensions

I just refactored the code a little bit to generate matrices (and a kernel) with more dimensions.

In [4]:
assert ConwayCubes(teststate, 4).run() == 848

In [5]:
print("Part 1:", ConwayCubes(initial_state, 4).run())

Part 1: 1792
